[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Digital-AI-Finance/Digital-Finance-Introduction/blob/main/day_05/notebooks/NB11_DeFi_Exploits.ipynb)

# NB11: DeFi Exploits and Security

**Topic:** 5.1 - DeFi Security and Risk Management

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand Common Attack Vectors**: Identify and explain the main types of DeFi exploits
2. **Analyze Historical Exploits**: Study real-world cases and understand what went wrong
3. **Simulate Reentrancy Attacks**: Understand the mechanics through educational Python simulations
4. **Understand Flash Loan Attacks**: Learn how uncollateralized loans enable complex exploits
5. **Learn Security Best Practices**: Apply defensive patterns to prevent vulnerabilities

---

**IMPORTANT DISCLAIMER**: This notebook is for **DEFENSIVE EDUCATION ONLY**. Understanding vulnerabilities is essential for:
- Smart contract developers building secure protocols
- Security auditors identifying weaknesses
- DeFi users understanding risks before investing
- Regulators assessing systemic risks

The code examples are **simplified simulations in Python** that demonstrate concepts. They are NOT usable exploit code.

## Section 1: Setup

### What This Cell Does
This cell installs the Python libraries needed for the notebook (like pandas for data tables and matplotlib for charts).

**What to observe:** You should see installation messages. Once complete, you'll see "Successfully installed" messages.

**You don't need to understand the command — just run the cell and wait for it to complete.**

### What This Cell Does
This cell installs the Python libraries needed for the notebook (like pandas for data tables and matplotlib for charts).

**What to observe:** You should see installation messages. Once complete, you'll see "Successfully installed" messages.

**You don't need to understand the command — just run the cell and wait for it to complete.**

### What This Cell Does
This cell loads the installed libraries so we can use them in the rest of the notebook. Think of it like opening apps on your computer.

**What to observe:** You should see "Libraries loaded successfully!" followed by version numbers.

**You don't need to understand the code — just run it and look for the success message.**

In [ ]:
# Install required packages
!pip install -q pandas numpy plotly matplotlib

### What This Cell Does
This cell loads the installed libraries so we can use them in the rest of the notebook. Think of it like opening apps on your computer.

**What to observe:** You should see "Libraries loaded successfully!" followed by version numbers.

**You don't need to understand the code — just run it and look for the success message.**

### What This Cell Does
This cell creates a dataset of real DeFi exploits from 2016-2023 and displays it as a table.

**What to observe:** A table showing dates, protocol names, amounts stolen, and attack types. Look at which exploits were the largest and which types of attacks are most common.

**Try noticing:** Which year had the most exploits? Which type of attack caused the most total losses?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Callable
from dataclasses import dataclass, field
import time
import hashlib

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print("Libraries loaded successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

### What This Cell Does
This cell calculates summary statistics: total losses by attack type and by year.

**What to observe:** Two tables showing which attack types are most costly and which years had the most incidents.

**Try noticing:** Is there a trend over time? Are exploits getting larger or more frequent?

## Section 2: DeFi Security Landscape

### The Scale of DeFi Exploits

DeFi (Decentralized Finance) has experienced explosive growth since 2020, but with that growth has come significant security incidents. Let's examine the data:

### What This Cell Does
This cell creates visual charts showing the top 10 largest exploits and a pie chart of losses by attack type.

**What to observe:** Two colorful charts that make the data easier to understand visually.

**Try noticing:** Which single exploit was the largest? What percentage of losses come from bridge attacks?

### What This Cell Does
This cell creates a dataset of real DeFi exploits from 2016-2023 and displays it as a table.

**What to observe:** A table showing dates, protocol names, amounts stolen, and attack types. Look at which exploits were the largest and which types of attacks are most common.

**Try noticing:** Which year had the most exploits? Which type of attack caused the most total losses?

In [ ]:
# Major DeFi exploits dataset (values in millions USD)
major_exploits = [
    {"date": "2016-06-17", "protocol": "The DAO", "amount_m": 60, "type": "Reentrancy", "chain": "Ethereum"},
    {"date": "2020-02-15", "protocol": "bZx", "amount_m": 8, "type": "Flash Loan + Oracle", "chain": "Ethereum"},
    {"date": "2020-11-14", "protocol": "Akropolis", "amount_m": 2, "type": "Reentrancy", "chain": "Ethereum"},
    {"date": "2021-02-04", "protocol": "Yearn Finance", "amount_m": 11, "type": "Flash Loan", "chain": "Ethereum"},
    {"date": "2021-03-08", "protocol": "DODO", "amount_m": 3.8, "type": "Flash Loan", "chain": "Ethereum"},
    {"date": "2021-05-02", "protocol": "Spartan Protocol", "amount_m": 30, "type": "Flash Loan", "chain": "BSC"},
    {"date": "2021-05-20", "protocol": "PancakeBunny", "amount_m": 45, "type": "Flash Loan + Oracle", "chain": "BSC"},
    {"date": "2021-08-10", "protocol": "Poly Network", "amount_m": 611, "type": "Access Control", "chain": "Multi-chain"},
    {"date": "2021-10-27", "protocol": "Cream Finance", "amount_m": 130, "type": "Flash Loan + Oracle", "chain": "Ethereum"},
    {"date": "2021-12-02", "protocol": "BadgerDAO", "amount_m": 120, "type": "Frontend Attack", "chain": "Ethereum"},
    {"date": "2022-02-02", "protocol": "Wormhole", "amount_m": 320, "type": "Signature Verification", "chain": "Solana"},
    {"date": "2022-03-29", "protocol": "Ronin Bridge", "amount_m": 625, "type": "Private Key Compromise", "chain": "Ronin"},
    {"date": "2022-04-17", "protocol": "Beanstalk", "amount_m": 182, "type": "Governance Attack", "chain": "Ethereum"},
    {"date": "2022-06-24", "protocol": "Harmony Bridge", "amount_m": 100, "type": "Private Key Compromise", "chain": "Harmony"},
    {"date": "2022-08-02", "protocol": "Nomad Bridge", "amount_m": 190, "type": "Validation Bug", "chain": "Multi-chain"},
    {"date": "2022-10-06", "protocol": "BNB Bridge", "amount_m": 570, "type": "Proof Verification", "chain": "BSC"},
    {"date": "2022-11-11", "protocol": "FTX/Alameda", "amount_m": 477, "type": "Insider Fraud", "chain": "Multi-chain"},
    {"date": "2023-03-13", "protocol": "Euler Finance", "amount_m": 197, "type": "Flash Loan + Logic", "chain": "Ethereum"},
    {"date": "2023-07-30", "protocol": "Curve Finance", "amount_m": 70, "type": "Reentrancy (Vyper)", "chain": "Ethereum"},
    {"date": "2023-09-12", "protocol": "Stake.com", "amount_m": 41, "type": "Private Key Leak", "chain": "Multi-chain"},
]

df_exploits = pd.DataFrame(major_exploits)
df_exploits['date'] = pd.to_datetime(df_exploits['date'])
df_exploits['year'] = df_exploits['date'].dt.year

print("Major DeFi Exploits (2016-2023):")
print("=" * 80)
print(df_exploits.to_string(index=False))
print("\n" + "=" * 80)
print(f"\nTotal losses tracked: ${df_exploits['amount_m'].sum():,.0f} million USD")

### What This Cell Does
This cell displays a structured list of different types of DeFi attacks, organized by category.

**What to observe:** Three main categories of attacks with descriptions of each type.

**You don't need to memorize these — just run the cell to see the taxonomy.**

### What This Cell Does
This cell calculates summary statistics: total losses by attack type and by year.

**What to observe:** Two tables showing which attack types are most costly and which years had the most incidents.

**Try noticing:** Is there a trend over time? Are exploits getting larger or more frequent?

In [ ]:
# Analyze by exploit type
print("\nLosses by Attack Type:")
print("=" * 60)
by_type = df_exploits.groupby('type')['amount_m'].agg(['sum', 'count', 'mean']).round(1)
by_type.columns = ['Total ($M)', 'Count', 'Avg ($M)']
by_type = by_type.sort_values('Total ($M)', ascending=False)
print(by_type)

print("\n" + "=" * 60)
print("\nLosses by Year:")
print("=" * 60)
by_year = df_exploits.groupby('year')['amount_m'].agg(['sum', 'count']).round(1)
by_year.columns = ['Total ($M)', 'Incidents']
print(by_year)

### What This Cell Does
This cell defines a simulated "vulnerable bank" that has a security bug. The bug is that it sends money BEFORE updating account balances.

**What to observe:** After running this cell, nothing visible happens yet. It's creating the simulation that we'll run in the next cell.

**You don't need to understand the code — this is setting up a demonstration. The code contains intentional bugs for educational purposes.**

### What This Cell Does
This cell creates visual charts showing the top 10 largest exploits and a pie chart of losses by attack type.

**What to observe:** Two colorful charts that make the data easier to understand visually.

**Try noticing:** Which single exploit was the largest? What percentage of losses come from bridge attacks?

### What This Cell Does
This cell creates a simulated "attacker" that can exploit the vulnerability in the bank from the previous cell.

**What to observe:** Nothing visible yet — this is more setup for the demonstration.

**You don't need to understand the code — just run it. The next cell will show the attack in action.**

In [ ]:
# Visualize the data
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Top 10 largest exploits
top_10 = df_exploits.nlargest(10, 'amount_m')
colors = plt.cm.Reds(np.linspace(0.4, 0.9, len(top_10)))
axes[0].barh(range(len(top_10)), top_10['amount_m'], color=colors)
axes[0].set_yticks(range(len(top_10)))
axes[0].set_yticklabels([f"{row['protocol']} ({row['date'].strftime('%Y')})" for _, row in top_10.iterrows()])
axes[0].set_xlabel('Amount Lost ($ millions)')
axes[0].set_title('Top 10 Largest DeFi Exploits')
axes[0].invert_yaxis()

# Add value labels
for i, v in enumerate(top_10['amount_m']):
    axes[0].text(v + 5, i, f'${v:.0f}M', va='center', fontsize=9)

# Right: Losses by attack type
type_totals = by_type['Total ($M)'].head(6)
colors = plt.cm.tab10(np.arange(len(type_totals)))
wedges, texts, autotexts = axes[1].pie(
    type_totals, 
    labels=type_totals.index, 
    autopct='%1.1f%%',
    colors=colors,
    pctdistance=0.75
)
axes[1].set_title('Losses by Attack Type')

plt.tight_layout()
plt.show()

print("\nKey Insights:")
print("1. Bridge attacks have been the most costly category")
print("2. Flash loan attacks are the most frequent vector")
print("3. 2022 was the worst year for DeFi exploits")
print("4. Multi-chain protocols face unique security challenges")

### What This Cell Does
This cell demonstrates a reentrancy attack by running the simulation. You'll see legitimate users deposit money, then an attacker exploits the vulnerability to steal it.

**What to observe:**
- Legitimate users (Alice, Bob, Charlie) deposit money
- The attacker deposits only 1 ETH but steals much more
- Watch how the attacker's callback function gets called repeatedly

**Key insight:** The attacker withdrew money 5 times even though they only had a balance of 1 ETH! This is because the bank updated balances AFTER sending money.

### Attack Vector Taxonomy

DeFi exploits can be categorized into several main types:

### What This Cell Does
This cell displays a structured list of different types of DeFi attacks, organized by category.

**What to observe:** Three main categories of attacks with descriptions of each type.

**You don't need to memorize these — just run the cell to see the taxonomy.**

### What This Cell Does
This cell creates a SECURE version of the bank that fixes the vulnerability using two protective techniques.

**What to observe:** Nothing visible yet — this is setting up the secure version.

**You don't need to understand the code — the next cell will show how the secure version blocks the attack.**

In [ ]:
attack_vectors = {
    "Smart Contract Vulnerabilities": {
        "Reentrancy": "Contract calls external address before updating state",
        "Integer Overflow/Underflow": "Arithmetic errors causing unexpected values",
        "Access Control": "Missing or improper permission checks",
        "Logic Errors": "Flawed business logic in contract code",
        "Uninitialized Storage": "Variables not properly initialized"
    },
    "Economic Exploits": {
        "Flash Loan Attacks": "Using uncollateralized loans to manipulate markets",
        "Oracle Manipulation": "Manipulating price feeds to exploit protocols",
        "Sandwich Attacks": "Front/back-running transactions for profit",
        "Governance Attacks": "Exploiting voting mechanisms"
    },
    "Infrastructure Attacks": {
        "Bridge Exploits": "Attacking cross-chain bridges",
        "Frontend Attacks": "Compromising web interfaces",
        "Private Key Compromise": "Stealing administrative keys",
        "DNS Hijacking": "Redirecting users to malicious sites"
    }
}

print("DeFi Attack Vector Taxonomy")
print("=" * 80)

for category, vectors in attack_vectors.items():
    print(f"\n{category}:")
    print("-" * 40)
    for name, description in vectors.items():
        print(f"  {name}:")
        print(f"    {description}")

### What This Cell Does
This cell demonstrates that the same attack FAILS against the secure bank.

**What to observe:**
- Legitimate users deposit money (same as before)
- The attacker tries the same exploit
- This time, the reentrancy guard BLOCKS the attack
- User funds remain safe!

**Key insight:** The message "BLOCKED: Reentrancy attempt" shows the defense working. The secure bank updates balances FIRST, then makes external calls.

## Section 3: Reentrancy Attack Simulation

### What is a Reentrancy Attack?

A **reentrancy attack** occurs when a contract calls an external address (e.g., to send ETH), and that external address calls back into the original contract before the first execution completes.

The classic pattern:
1. Vulnerable contract has a `withdraw()` function
2. It sends ETH to the caller BEFORE updating the balance
3. Attacker's contract receives ETH and immediately calls `withdraw()` again
4. The balance hasn't been updated yet, so the attacker can withdraw again
5. This repeats until the contract is drained

Let's simulate this in Python:

### What This Cell Does
This cell defines a simulated "vulnerable bank" that has a security bug. The bug is that it sends money BEFORE updating account balances.

**What to observe:** After running this cell, nothing visible happens yet. It's creating the simulation that we'll run in the next cell.

**You don't need to understand the code — this is setting up a demonstration. The code contains intentional bugs for educational purposes.**

### What This Cell Does
This cell creates a simulation of a flash loan provider (like Aave). Flash loans let you borrow huge amounts with zero collateral, but you must repay in the same transaction.

**What to observe:** Nothing visible yet — this sets up the flash loan system.

**You don't need to understand the code — the next cells will demonstrate both legitimate and malicious uses of flash loans.**

In [ ]:
@dataclass
class Account:
    """Represents an account with a balance."""
    address: str
    balance: float = 0.0

class VulnerableBank:
    """
    A VULNERABLE smart contract simulation.
    
    THIS CODE DEMONSTRATES A SECURITY VULNERABILITY.
    The bug: It sends funds BEFORE updating the balance.
    """
    
    def __init__(self, name: str = "VulnerableBank"):
        self.name = name
        self.balances: Dict[str, float] = {}
        self.total_balance: float = 0.0
        self.call_depth: int = 0
        self.max_call_depth: int = 0
        self.withdrawal_log: List[Dict] = []
        
    def deposit(self, sender: str, amount: float) -> None:
        """Deposit funds into the bank."""
        if amount <= 0:
            raise ValueError("Deposit must be positive")
        
        self.balances[sender] = self.balances.get(sender, 0) + amount
        self.total_balance += amount
        print(f"  [DEPOSIT] {sender} deposited {amount:.2f} ETH")
        
    def withdraw(self, sender: str, callback: Optional[Callable] = None) -> float:
        """
        VULNERABLE withdraw function.
        
        BUG: Sends ETH before updating balance!
        """
        self.call_depth += 1
        self.max_call_depth = max(self.max_call_depth, self.call_depth)
        
        balance = self.balances.get(sender, 0)
        
        if balance > 0:
            print(f"  [WITHDRAW depth={self.call_depth}] {sender} withdrawing {balance:.2f} ETH")
            print(f"    Current balances[{sender}] = {self.balances.get(sender, 0):.2f}")
            
            # BUG: Send funds BEFORE updating balance
            # In Solidity: msg.sender.call{value: balance}("")
            self.withdrawal_log.append({
                'sender': sender,
                'amount': balance,
                'depth': self.call_depth,
                'balance_before_update': self.balances.get(sender, 0)
            })
            
            # This is where the vulnerability occurs!
            # The callback (simulating receiving ETH) is called BEFORE balance update
            if callback and self.total_balance > 0:
                print(f"    --> Calling back to attacker... (balance not yet updated!)")
                callback()  # Attacker's fallback function is called
            
            # BUG: Balance is updated AFTER the external call
            # By this point, the attacker may have already withdrawn multiple times!
            self.balances[sender] = 0
            self.total_balance -= balance
            print(f"    Balance updated: balances[{sender}] = 0, total = {self.total_balance:.2f}")
            
            self.call_depth -= 1
            return balance
        
        self.call_depth -= 1
        return 0
    
    def get_balance(self, address: str) -> float:
        """Get balance of an address."""
        return self.balances.get(address, 0)
    
    def status(self) -> None:
        """Print bank status."""
        print(f"\n{self.name} Status:")
        print(f"  Total Balance: {self.total_balance:.2f} ETH")
        print(f"  User Balances: {dict(self.balances)}")

### What This Cell Does
This cell creates simulated DEX (Decentralized Exchange) systems where tokens can be traded.

**What to observe:** Nothing visible yet — this sets up the trading platforms.

**You don't need to understand the code — these are just tools for the demonstrations in the next cells.**

### What This Cell Does
This cell creates a simulated "attacker" that can exploit the vulnerability in the bank from the previous cell.

**What to observe:** Nothing visible yet — this is more setup for the demonstration.

**You don't need to understand the code — just run it. The next cell will show the attack in action.**

### What This Cell Does
This cell demonstrates a LEGITIMATE use of flash loans: arbitrage between two exchanges with different prices.

**What to observe:**
- Two DEXes with different prices for the same token
- A trader borrows money via flash loan, buys low, sells high, repays the loan, and keeps the profit
- The prices on both DEXes become closer (more efficient market)

**Key insight:** Flash loans enable arbitrage without requiring upfront capital. This actually helps markets by equalizing prices!

In [ ]:
class ReentrancyAttacker:
    """
    Attacker contract that exploits the reentrancy vulnerability.
    
    EDUCATIONAL PURPOSE ONLY - This demonstrates the attack pattern.
    """
    
    def __init__(self, name: str, target: VulnerableBank, max_attacks: int = 5):
        self.name = name
        self.target = target
        self.stolen_funds: float = 0.0
        self.attack_count: int = 0
        self.max_attacks = max_attacks
        
    def attack(self, initial_deposit: float) -> float:
        """
        Execute the reentrancy attack.
        
        1. Deposit a small amount
        2. Call withdraw()
        3. In the callback (simulating receive()), call withdraw() again
        4. Repeat until drained or max attacks reached
        """
        print(f"\n{'='*70}")
        print(f"REENTRANCY ATTACK SIMULATION")
        print(f"{'='*70}")
        print(f"Attacker: {self.name}")
        print(f"Target: {self.target.name}")
        print(f"Target balance before: {self.target.total_balance:.2f} ETH")
        print(f"Initial deposit: {initial_deposit:.2f} ETH")
        print(f"{'='*70}")
        
        # Step 1: Make a small deposit to become a valid user
        self.target.deposit(self.name, initial_deposit)
        
        print(f"\n--- Starting attack ---")
        
        # Step 2: Call withdraw with our malicious callback
        self.target.withdraw(self.name, callback=self._malicious_callback)
        
        print(f"\n{'='*70}")
        print(f"ATTACK COMPLETE")
        print(f"{'='*70}")
        print(f"Attack iterations: {self.attack_count}")
        print(f"Max call depth reached: {self.target.max_call_depth}")
        print(f"Funds stolen: {self.stolen_funds:.2f} ETH")
        print(f"Target balance after: {self.target.total_balance:.2f} ETH")
        print(f"{'='*70}")
        
        return self.stolen_funds
        
    def _malicious_callback(self) -> None:
        """
        This simulates the attacker's fallback/receive function.
        It gets called when the vulnerable contract sends ETH.
        
        In Solidity, this would be:
        receive() external payable {
            if (address(target).balance >= 1 ether) {
                target.withdraw();
            }
        }
        """
        self.attack_count += 1
        amount = self.target.get_balance(self.name)
        self.stolen_funds += amount
        
        print(f"    [CALLBACK #{self.attack_count}] Received ETH! Stolen so far: {self.stolen_funds:.2f}")
        
        # Keep attacking while there's money and we haven't hit our limit
        if self.target.total_balance > 0 and self.attack_count < self.max_attacks:
            print(f"    [CALLBACK] Re-entering withdraw()...")
            self.target.withdraw(self.name, callback=self._malicious_callback)

### What This Cell Does
This cell demonstrates a reentrancy attack by running the simulation. You'll see legitimate users deposit money, then an attacker exploits the vulnerability to steal it.

**What to observe:**
- Legitimate users (Alice, Bob, Charlie) deposit money
- The attacker deposits only 1 ETH but steals much more
- Watch how the attacker's callback function gets called repeatedly

**Key insight:** The attacker withdrew money 5 times even though they only had a balance of 1 ETH! This is because the bank updated balances AFTER sending money.

### What This Cell Does
This cell creates simulations of a vulnerable price oracle and a lending protocol that relies on it.

**What to observe:** Nothing visible yet — this sets up a system with a security flaw.

**You don't need to understand the code — the next cell will demonstrate the attack.**

In [ ]:
# Demonstrate the attack
print("REENTRANCY ATTACK DEMONSTRATION")
print("=" * 70)

# Create a vulnerable bank with funds from legitimate users
bank = VulnerableBank("VulnerableBank")

# Legitimate users deposit funds
print("\n--- Legitimate users depositing ---")
bank.deposit("Alice", 10.0)
bank.deposit("Bob", 15.0)
bank.deposit("Charlie", 25.0)

bank.status()

# Attacker strikes
attacker = ReentrancyAttacker("Attacker", bank, max_attacks=5)
stolen = attacker.attack(initial_deposit=1.0)  # Only deposit 1 ETH

# Final status
print("\n--- Final State ---")
bank.status()
print(f"\nAttacker profit: {stolen - 1.0:.2f} ETH (stolen {stolen:.2f} - deposited 1.0)")
print(f"Legitimate users lost: {50.0 - bank.total_balance - stolen + 1:.2f} ETH")

### What This Cell Does
This cell demonstrates an oracle manipulation attack where an attacker uses a large trade to temporarily inflate prices and borrow more than their collateral is worth.

**What to observe:**
- Initial fair price (~100)
- Attacker manipulates DEX to inflate price to ~164
- Attacker borrows at inflated price
- Attacker reverses manipulation
- Result: Attacker profits, protocol has bad debt

**Key insight:** Using spot prices (current market prices) is dangerous because they can be manipulated with flash loans!

### The Fix: Checks-Effects-Interactions Pattern

The secure pattern is to:
1. **Check**: Validate all conditions
2. **Effect**: Update state (balances)
3. **Interact**: Make external calls LAST

Let's implement a secure version:

### What This Cell Does
This cell creates a SECURE version of the bank that fixes the vulnerability using two protective techniques.

**What to observe:** Nothing visible yet — this is setting up the secure version.

**You don't need to understand the code — the next cell will show how the secure version blocks the attack.**

### What This Cell Does
This cell demonstrates how Time-Weighted Average Price (TWAP) oracles resist manipulation.

**What to observe:**
- Historical prices around 100
- Attacker manipulates spot price to 300
- TWAP only moves slightly (to ~120) instead of jumping to 300
- The attacker would need to maintain the manipulation for many blocks (very expensive!)

**Key insight:** TWAP uses historical prices instead of just the current price, making manipulation much harder and more expensive.

In [ ]:
class SecureBank:
    """
    A SECURE smart contract simulation.
    
    Uses the Checks-Effects-Interactions pattern.
    Also implements a reentrancy guard for extra protection.
    """
    
    def __init__(self, name: str = "SecureBank"):
        self.name = name
        self.balances: Dict[str, float] = {}
        self.total_balance: float = 0.0
        self._locked: bool = False  # Reentrancy guard
        self.call_attempts: List[Dict] = []
        
    def deposit(self, sender: str, amount: float) -> None:
        """Deposit funds into the bank."""
        if amount <= 0:
            raise ValueError("Deposit must be positive")
        
        self.balances[sender] = self.balances.get(sender, 0) + amount
        self.total_balance += amount
        print(f"  [DEPOSIT] {sender} deposited {amount:.2f} ETH")
        
    def withdraw(self, sender: str, callback: Optional[Callable] = None) -> float:
        """
        SECURE withdraw function using Checks-Effects-Interactions.
        
        1. CHECK: Verify conditions
        2. EFFECT: Update state FIRST
        3. INTERACT: External call LAST
        """
        # Reentrancy guard
        if self._locked:
            print(f"  [BLOCKED] Reentrancy attempt from {sender}!")
            self.call_attempts.append({'sender': sender, 'blocked': True})
            return 0
        
        self._locked = True
        
        try:
            # CHECKS
            balance = self.balances.get(sender, 0)
            if balance <= 0:
                print(f"  [WITHDRAW] {sender} has no balance")
                return 0
            
            print(f"  [WITHDRAW] {sender} withdrawing {balance:.2f} ETH")
            
            # EFFECTS - Update state BEFORE external call
            self.balances[sender] = 0
            self.total_balance -= balance
            print(f"    State updated FIRST: balance = 0, total = {self.total_balance:.2f}")
            
            # INTERACTIONS - External call LAST
            if callback:
                print(f"    --> Making external call (balance already updated!)")
                callback()
            
            return balance
            
        finally:
            self._locked = False
    
    def get_balance(self, address: str) -> float:
        """Get balance of an address."""
        return self.balances.get(address, 0)
    
    def status(self) -> None:
        """Print bank status."""
        print(f"\n{self.name} Status:")
        print(f"  Total Balance: {self.total_balance:.2f} ETH")
        print(f"  User Balances: {dict(self.balances)}")
        if self.call_attempts:
            print(f"  Blocked reentrancy attempts: {len([c for c in self.call_attempts if c['blocked']])}")

### What This Cell Does
This cell demonstrates that the same attack FAILS against the secure bank.

**What to observe:**
- Legitimate users deposit money (same as before)
- The attacker tries the same exploit
- This time, the reentrancy guard BLOCKS the attack
- User funds remain safe!

**Key insight:** The message "BLOCKED: Reentrancy attempt" shows the defense working. The secure bank updates balances FIRST, then makes external calls.

### What This Cell Does
This cell displays information about The DAO Hack (2016), the first major DeFi exploit.

**What to observe:** A detailed description of what happened, how the attack worked, and the aftermath (Ethereum hard fork).

**Key insight:** This was a reentrancy attack similar to what we simulated earlier. It led to the Ethereum/Ethereum Classic split.

In [ ]:
# Test the secure version
print("SECURE BANK - ATTACK ATTEMPT")
print("=" * 70)

# Create secure bank with funds
secure_bank = SecureBank("SecureBank")

print("\n--- Legitimate users depositing ---")
secure_bank.deposit("Alice", 10.0)
secure_bank.deposit("Bob", 15.0)
secure_bank.deposit("Charlie", 25.0)

secure_bank.status()

# Create attacker targeting secure bank
class AttackerAgainstSecure:
    def __init__(self, name: str, target: SecureBank):
        self.name = name
        self.target = target
        self.attack_attempts = 0
        
    def attack(self, initial_deposit: float):
        print(f"\n{'='*70}")
        print(f"ATTEMPTING ATTACK ON SECURE BANK")
        print(f"{'='*70}")
        
        self.target.deposit(self.name, initial_deposit)
        
        print(f"\n--- Starting attack ---")
        self.target.withdraw(self.name, callback=self._malicious_callback)
        
        print(f"\n{'='*70}")
        print(f"ATTACK FAILED!")
        print(f"{'='*70}")
        print(f"Attack attempts: {self.attack_attempts}")
        
    def _malicious_callback(self):
        self.attack_attempts += 1
        print(f"    [CALLBACK #{self.attack_attempts}] Trying to re-enter...")
        self.target.withdraw(self.name, callback=self._malicious_callback)

# Try the attack
attacker2 = AttackerAgainstSecure("Attacker", secure_bank)
attacker2.attack(1.0)

print("\n--- Final State ---")
secure_bank.status()
print(f"\nUser funds are SAFE! The reentrancy guard blocked the attack.")

### What This Cell Does
This cell displays information about the Ronin Bridge Hack (2022), one of the largest crypto exploits ever.

**What to observe:** Description of how North Korean hackers compromised validator keys to steal $625 million.

**Key insight:** This wasn't a code bug — it was a social engineering attack that compromised private keys. Decentralization matters!

## Section 4: Flash Loan Mechanics

### What are Flash Loans?

**Flash loans** are uncollateralized loans that must be borrowed and repaid within a single transaction (single block). If not repaid, the entire transaction reverts.

Key properties:
- **Zero collateral**: No upfront capital needed
- **Atomic**: Borrow and repay in same transaction or everything reverts
- **Large amounts**: Can borrow millions of dollars
- **Low cost**: Only pay a small fee (0.05-0.09%)

Legitimate uses:
- Arbitrage between DEXes
- Collateral swaps
- Self-liquidation

Malicious uses:
- Market manipulation
- Oracle attacks
- Governance attacks

### What This Cell Does
This cell displays information about the Wormhole Bridge Hack (2022).

**What to observe:** Description of how a signature verification bug allowed forging messages to mint tokens without real deposits.

**Key insight:** Cross-chain bridges are complex and vulnerable. A fix had been prepared but not deployed yet!

### What This Cell Does
This cell creates a simulation of a flash loan provider (like Aave). Flash loans let you borrow huge amounts with zero collateral, but you must repay in the same transaction.

**What to observe:** Nothing visible yet — this sets up the flash loan system.

**You don't need to understand the code — the next cells will demonstrate both legitimate and malicious uses of flash loans.**

In [ ]:
@dataclass
class FlashLoanProvider:
    """
    Simulates a flash loan provider like Aave or dYdX.
    """
    name: str
    liquidity: float
    fee_rate: float = 0.0009  # 0.09% (Aave's fee)
    
    def flash_loan(
        self, 
        borrower: str, 
        amount: float, 
        operation: Callable
    ) -> Dict:
        """
        Execute a flash loan.
        
        The operation must return the amount to repay (principal + fee).
        If insufficient funds returned, transaction reverts.
        """
        print(f"\n--- Flash Loan from {self.name} ---")
        print(f"Borrower: {borrower}")
        print(f"Amount: {amount:,.2f}")
        
        if amount > self.liquidity:
            raise ValueError(f"Insufficient liquidity. Max: {self.liquidity:,.2f}")
        
        fee = amount * self.fee_rate
        required_repayment = amount + fee
        
        print(f"Fee ({self.fee_rate*100:.2f}%): {fee:,.2f}")
        print(f"Must repay: {required_repayment:,.2f}")
        
        # Execute the borrower's operation
        print(f"\n>>> Executing operation...")
        
        try:
            profit = operation()
            
            if profit >= required_repayment:
                net_profit = profit - required_repayment
                print(f"\n>>> Operation successful!")
                print(f"Profit from operation: {profit:,.2f}")
                print(f"Repayment: {required_repayment:,.2f}")
                print(f"Net profit: {net_profit:,.2f}")
                return {'success': True, 'profit': net_profit, 'amount': amount}
            else:
                raise ValueError(f"Insufficient repayment: {profit:,.2f} < {required_repayment:,.2f}")
                
        except Exception as e:
            print(f"\n>>> TRANSACTION REVERTED!")
            print(f"Reason: {e}")
            return {'success': False, 'error': str(e), 'amount': amount}

### What This Cell Does
This cell displays a comprehensive guide to smart contract security patterns with code examples.

**What to observe:** Six major security patterns with descriptions and example code showing both vulnerable and secure approaches.

**You don't need to understand the Solidity code — focus on reading the descriptions and understanding the concepts.**

### What This Cell Does
This cell creates simulated DEX (Decentralized Exchange) systems where tokens can be traded.

**What to observe:** Nothing visible yet — this sets up the trading platforms.

**You don't need to understand the code — these are just tools for the demonstrations in the next cells.**

In [ ]:
# Simulate a simple DEX for our examples
@dataclass
class SimpleDEX:
    """Simple AMM DEX for demonstration."""
    name: str
    token_a_reserve: float
    token_b_reserve: float
    fee_rate: float = 0.003  # 0.3%
    
    def get_price(self) -> float:
        """Get price of token A in terms of token B."""
        return self.token_b_reserve / self.token_a_reserve
    
    def swap_a_for_b(self, amount_a: float) -> float:
        """Swap token A for token B using x*y=k formula."""
        k = self.token_a_reserve * self.token_b_reserve
        
        # Apply fee
        amount_a_after_fee = amount_a * (1 - self.fee_rate)
        
        new_a_reserve = self.token_a_reserve + amount_a_after_fee
        new_b_reserve = k / new_a_reserve
        
        amount_b_out = self.token_b_reserve - new_b_reserve
        
        self.token_a_reserve = new_a_reserve
        self.token_b_reserve = new_b_reserve
        
        return amount_b_out
    
    def swap_b_for_a(self, amount_b: float) -> float:
        """Swap token B for token A."""
        k = self.token_a_reserve * self.token_b_reserve
        
        amount_b_after_fee = amount_b * (1 - self.fee_rate)
        
        new_b_reserve = self.token_b_reserve + amount_b_after_fee
        new_a_reserve = k / new_b_reserve
        
        amount_a_out = self.token_a_reserve - new_a_reserve
        
        self.token_a_reserve = new_a_reserve
        self.token_b_reserve = new_b_reserve
        
        return amount_a_out

### What This Cell Does
This cell displays the complete process that security auditors follow when reviewing smart contracts.

**What to observe:** Six phases from preparation through remediation, with specific tasks in each phase.

**Key insight:** Auditing is systematic and multi-layered, combining automated tools with manual expert review.

### What This Cell Does
This cell demonstrates a LEGITIMATE use of flash loans: arbitrage between two exchanges with different prices.

**What to observe:**
- Two DEXes with different prices for the same token
- A trader borrows money via flash loan, buys low, sells high, repays the loan, and keeps the profit
- The prices on both DEXes become closer (more efficient market)

**Key insight:** Flash loans enable arbitrage without requiring upfront capital. This actually helps markets by equalizing prices!

### What This Cell Does
This cell displays how audit findings are classified by severity level (Critical, High, Medium, Low, Informational).

**What to observe:** Each severity level with description, examples, and recommended actions.

**Key insight:** Not all security issues are equally urgent. Critical issues must be fixed before deployment.**

In [ ]:
# Demonstrate legitimate flash loan arbitrage
print("LEGITIMATE FLASH LOAN ARBITRAGE")
print("=" * 70)

# Two DEXes with different prices
dex_a = SimpleDEX("DEX_A", token_a_reserve=1000, token_b_reserve=100000)  # Price: 100
dex_b = SimpleDEX("DEX_B", token_a_reserve=1000, token_b_reserve=110000)  # Price: 110

print(f"\nInitial state:")
print(f"  {dex_a.name}: Price = {dex_a.get_price():.2f} (Token A = {dex_a.token_a_reserve}, Token B = {dex_a.token_b_reserve})")
print(f"  {dex_b.name}: Price = {dex_b.get_price():.2f} (Token A = {dex_b.token_a_reserve}, Token B = {dex_b.token_b_reserve})")
print(f"  Price difference: {((dex_b.get_price() / dex_a.get_price()) - 1) * 100:.2f}%")

# Flash loan provider
flash_provider = FlashLoanProvider("Aave", liquidity=1000000)

def arbitrage_operation() -> float:
    """Execute arbitrage between two DEXes."""
    flash_amount = 10000  # Borrow 10,000 Token B
    
    print(f"\n  Step 1: Borrow {flash_amount:,.0f} Token B")
    
    print(f"  Step 2: Buy Token A on {dex_a.name} (cheaper)")
    token_a_bought = dex_a.swap_b_for_a(flash_amount)
    print(f"           Got {token_a_bought:.4f} Token A")
    
    print(f"  Step 3: Sell Token A on {dex_b.name} (more expensive)")
    token_b_received = dex_b.swap_a_for_b(token_a_bought)
    print(f"           Got {token_b_received:,.2f} Token B")
    
    profit = token_b_received
    print(f"  Step 4: Repay flash loan with profit")
    
    return profit

# Execute the arbitrage
result = flash_provider.flash_loan("Arbitrageur", 10000, arbitrage_operation)

print(f"\nFinal state:")
print(f"  {dex_a.name}: Price = {dex_a.get_price():.2f}")
print(f"  {dex_b.name}: Price = {dex_b.get_price():.2f}")
print(f"  Prices are now closer - arbitrage improves market efficiency!")

### What This Cell Does
This cell displays a reference guide to audit firms, automated security tools, bug bounty platforms, and resources.

**What to observe:** Lists of top audit firms, security tools, bug bounty platforms, and learning resources.

**You might want to:** Bookmark some of these resources for later exploration (like Rekt.news for exploit analysis).

## Section 5: Oracle Manipulation Simulation

### What is Oracle Manipulation?

Many DeFi protocols rely on **price oracles** to determine asset values. If an attacker can manipulate the oracle price, they can:
- Borrow more than their collateral is worth
- Liquidate positions unfairly
- Profit from price discrepancies

Common oracle attack patterns:
1. **Spot price manipulation**: Large trades that temporarily move prices
2. **TWAP manipulation**: Sustained attacks over multiple blocks
3. **Liquidity attacks**: Draining liquidity to make manipulation cheaper

### What This Cell Does
This cell creates simulations of a vulnerable price oracle and a lending protocol that relies on it.

**What to observe:** Nothing visible yet — this sets up a system with a security flaw.

**You don't need to understand the code — the next cell will demonstrate the attack.**

In [ ]:
class VulnerableOracle:
    """
    A vulnerable oracle that uses spot price from a single DEX.
    
    VULNERABILITY: Spot prices can be easily manipulated!
    """
    
    def __init__(self, name: str, dex: SimpleDEX):
        self.name = name
        self.dex = dex
        
    def get_price(self) -> float:
        """Get current spot price from DEX."""
        return self.dex.get_price()


class VulnerableLendingProtocol:
    """
    A lending protocol vulnerable to oracle manipulation.
    """
    
    def __init__(self, name: str, oracle: VulnerableOracle):
        self.name = name
        self.oracle = oracle
        self.collateral: Dict[str, float] = {}  # Token A deposited as collateral
        self.borrowed: Dict[str, float] = {}    # Token B borrowed
        self.liquidity: float = 100000          # Token B available to lend
        self.collateral_ratio: float = 1.5     # 150% collateralization required
        
    def deposit_collateral(self, user: str, amount: float) -> None:
        """Deposit Token A as collateral."""
        self.collateral[user] = self.collateral.get(user, 0) + amount
        print(f"  {user} deposited {amount:.2f} Token A as collateral")
        
    def borrow(self, user: str, amount: float) -> bool:
        """
        Borrow Token B against collateral.
        
        VULNERABILITY: Uses manipulable spot price!
        """
        collateral_amount = self.collateral.get(user, 0)
        current_price = self.oracle.get_price()  # Vulnerable: spot price
        collateral_value = collateral_amount * current_price
        
        already_borrowed = self.borrowed.get(user, 0)
        max_borrow = collateral_value / self.collateral_ratio - already_borrowed
        
        print(f"\n  Borrow attempt by {user}:")
        print(f"    Collateral: {collateral_amount:.2f} Token A")
        print(f"    Oracle price: {current_price:.2f}")
        print(f"    Collateral value: {collateral_value:,.2f} Token B")
        print(f"    Max borrow (150% ratio): {max_borrow:,.2f} Token B")
        print(f"    Requested: {amount:,.2f} Token B")
        
        if amount > max_borrow:
            print(f"    REJECTED: Exceeds borrowing capacity")
            return False
            
        if amount > self.liquidity:
            print(f"    REJECTED: Insufficient protocol liquidity")
            return False
        
        self.borrowed[user] = already_borrowed + amount
        self.liquidity -= amount
        print(f"    APPROVED: Borrowed {amount:,.2f} Token B")
        return True
    
    def get_position(self, user: str) -> Dict:
        """Get user's position details."""
        collateral = self.collateral.get(user, 0)
        borrowed = self.borrowed.get(user, 0)
        price = self.oracle.get_price()
        
        return {
            'collateral': collateral,
            'borrowed': borrowed,
            'collateral_value': collateral * price,
            'health_factor': (collateral * price) / borrowed if borrowed > 0 else float('inf')
        }

### What This Cell Does
This cell demonstrates an oracle manipulation attack where an attacker uses a large trade to temporarily inflate prices and borrow more than their collateral is worth.

**What to observe:**
- Initial fair price (~100)
- Attacker manipulates DEX to inflate price to ~164
- Attacker borrows at inflated price
- Attacker reverses manipulation
- Result: Attacker profits, protocol has bad debt

**Key insight:** Using spot prices (current market prices) is dangerous because they can be manipulated with flash loans!

In [ ]:
# Demonstrate oracle manipulation attack
print("ORACLE MANIPULATION ATTACK SIMULATION")
print("=" * 70)

# Setup
target_dex = SimpleDEX("TargetDEX", token_a_reserve=1000, token_b_reserve=100000)
vulnerable_oracle = VulnerableOracle("SpotPriceOracle", target_dex)
lending_protocol = VulnerableLendingProtocol("VulnerableLend", vulnerable_oracle)

print(f"\nInitial state:")
print(f"  DEX Price: {target_dex.get_price():.2f} Token B per Token A")
print(f"  Protocol liquidity: {lending_protocol.liquidity:,.2f} Token B")

# Attacker's plan:
# 1. Deposit some Token A as collateral
# 2. Manipulate DEX to inflate Token A price
# 3. Borrow more than collateral is actually worth
# 4. Keep the borrowed funds, abandon collateral

print(f"\n--- ATTACK BEGINS ---")

# Step 1: Deposit modest collateral
attacker_collateral = 50  # 50 Token A
lending_protocol.deposit_collateral("Attacker", attacker_collateral)

# Check borrowing power at normal price
print(f"\nAt normal price ({target_dex.get_price():.2f}):")
fair_value = attacker_collateral * target_dex.get_price()
print(f"  Fair collateral value: {fair_value:,.2f} Token B")
print(f"  Max borrow (normal): {fair_value / 1.5:,.2f} Token B")

# Step 2: Manipulate the DEX price with a large trade
print(f"\n--- MANIPULATING ORACLE ---")
manipulation_amount = 50000  # Pump 50,000 Token B into DEX
print(f"Attacker swaps {manipulation_amount:,} Token B for Token A on DEX...")

tokens_received = target_dex.swap_b_for_a(manipulation_amount)
print(f"  Received: {tokens_received:.2f} Token A")
print(f"  New DEX price: {target_dex.get_price():.2f} Token B per Token A")
print(f"  Price inflated by: {((target_dex.get_price() / 100) - 1) * 100:.1f}%")

# Step 3: Borrow at inflated price
print(f"\n--- EXPLOITING INFLATED PRICE ---")
inflated_value = attacker_collateral * target_dex.get_price()
print(f"  Inflated collateral value: {inflated_value:,.2f} Token B")

# Borrow maximum amount
borrow_amount = inflated_value / 1.5 - 100  # Leave small buffer
success = lending_protocol.borrow("Attacker", borrow_amount)

# Step 4: Reverse the manipulation (sell tokens back)
print(f"\n--- REVERSING MANIPULATION ---")
tokens_back = target_dex.swap_a_for_b(tokens_received)
print(f"  Sold {tokens_received:.2f} Token A back")
print(f"  Received: {tokens_back:,.2f} Token B")
print(f"  DEX price restored to: {target_dex.get_price():.2f}")

# Calculate attack profit
print(f"\n{'='*70}")
print(f"ATTACK RESULTS")
print(f"{'='*70}")

manipulation_cost = manipulation_amount - tokens_back
borrowed = lending_protocol.borrowed.get("Attacker", 0)
collateral_lost = attacker_collateral * 100  # At fair price

print(f"\nAttacker's gains:")
print(f"  Borrowed: {borrowed:,.2f} Token B")

print(f"\nAttacker's costs:")
print(f"  Collateral deposited (now worthless to attacker): {collateral_lost:,.2f} Token B equivalent")
print(f"  Manipulation slippage: {manipulation_cost:,.2f} Token B")

net_profit = borrowed - collateral_lost - manipulation_cost
print(f"\nNET PROFIT: {net_profit:,.2f} Token B")

print(f"\nProtocol loss:")
print(f"  Bad debt: {borrowed - collateral_lost:,.2f} Token B")
print(f"  (Collateral is worth less than borrowed amount at true price)")

### The Fix: Time-Weighted Average Price (TWAP) Oracle

Secure oracles use multiple data sources and time-weighted averages to resist manipulation:

### What This Cell Does
This cell demonstrates how Time-Weighted Average Price (TWAP) oracles resist manipulation.

**What to observe:**
- Historical prices around 100
- Attacker manipulates spot price to 300
- TWAP only moves slightly (to ~120) instead of jumping to 300
- The attacker would need to maintain the manipulation for many blocks (very expensive!)

**Key insight:** TWAP uses historical prices instead of just the current price, making manipulation much harder and more expensive.

In [ ]:
class SecureTWAPOracle:
    """
    A more secure oracle using Time-Weighted Average Price (TWAP).
    
    This is resistant to flash loan manipulation because:
    1. Uses historical prices, not just spot
    2. Requires manipulation over multiple blocks
    3. Can include multiple data sources
    """
    
    def __init__(self, name: str, window_size: int = 10):
        self.name = name
        self.window_size = window_size
        self.price_history: List[Tuple[float, float]] = []  # (timestamp, price)
        
    def update_price(self, price: float, timestamp: float = None) -> None:
        """Record a price observation."""
        if timestamp is None:
            timestamp = time.time()
        self.price_history.append((timestamp, price))
        
        # Keep only recent history
        if len(self.price_history) > self.window_size * 2:
            self.price_history = self.price_history[-self.window_size:]
    
    def get_twap(self) -> float:
        """Calculate time-weighted average price."""
        if len(self.price_history) < 2:
            return self.price_history[-1][1] if self.price_history else 0
        
        # Use last window_size observations
        recent = self.price_history[-self.window_size:]
        
        total_weighted = 0
        total_time = 0
        
        for i in range(1, len(recent)):
            time_delta = recent[i][0] - recent[i-1][0]
            price = recent[i-1][1]
            total_weighted += price * time_delta
            total_time += time_delta
        
        return total_weighted / total_time if total_time > 0 else recent[-1][1]
    
    def get_price(self) -> float:
        """Get the TWAP price."""
        return self.get_twap()


# Demonstrate TWAP resistance to manipulation
print("TWAP ORACLE RESISTANCE TO MANIPULATION")
print("=" * 70)

twap_oracle = SecureTWAPOracle("SecureTWAP", window_size=10)

# Build up price history (normal prices around 100)
base_time = time.time()
for i in range(10):
    normal_price = 100 + np.random.normal(0, 1)  # Slight variation
    twap_oracle.update_price(normal_price, base_time + i * 12)  # 12 seconds per block

print(f"Historical prices: {[f'{p[1]:.1f}' for p in twap_oracle.price_history]}")
print(f"Current TWAP: {twap_oracle.get_twap():.2f}")

# Attacker tries to manipulate with a single high price
print(f"\nAttacker manipulates spot price to 300...")
twap_oracle.update_price(300, base_time + 10 * 12)

print(f"New TWAP after manipulation: {twap_oracle.get_twap():.2f}")
print(f"Spot price showed 300, but TWAP only moved to {twap_oracle.get_twap():.2f}!")
print(f"\nThe attacker would need to maintain the manipulated price for many blocks")
print(f"to significantly affect the TWAP, which is expensive and risky.")

## Section 6: Case Studies

### Case Study 1: The DAO Hack (2016)

The **DAO Hack** was the first major DeFi exploit and led to the Ethereum hard fork.

### What This Cell Does
This cell displays information about The DAO Hack (2016), the first major DeFi exploit.

**What to observe:** A detailed description of what happened, how the attack worked, and the aftermath (Ethereum hard fork).

**Key insight:** This was a reentrancy attack similar to what we simulated earlier. It led to the Ethereum/Ethereum Classic split.

In [ ]:
dao_hack = {
    "date": "June 17, 2016",
    "protocol": "The DAO",
    "amount_stolen": "3.6 million ETH (~$60M at the time, ~$7B at 2021 peak)",
    "vulnerability": "Reentrancy",
    "description": """
The DAO was a decentralized venture capital fund that raised $150M in ETH.

The Attack:
1. The DAO had a 'splitDAO' function to withdraw funds
2. It sent ETH BEFORE updating the balance (classic reentrancy bug)
3. Attacker's contract received ETH and immediately called splitDAO again
4. This drained ~1/3 of the DAO's funds

The Vulnerable Code (simplified):
```solidity
function splitDAO() {
    // BUG: Sends ETH before updating balance
    msg.sender.call.value(balance)("");  // Attacker re-enters here!
    balances[msg.sender] = 0;  // Too late - already re-entered
}
```

The Aftermath:
- Ethereum community debated: let the hack stand or hard fork?
- A hard fork was executed to return the funds
- This created Ethereum (ETH) and Ethereum Classic (ETC)
- Established the importance of smart contract security
"""
}

print("CASE STUDY: THE DAO HACK")
print("=" * 70)
print(f"Date: {dao_hack['date']}")
print(f"Protocol: {dao_hack['protocol']}")
print(f"Amount: {dao_hack['amount_stolen']}")
print(f"Vulnerability: {dao_hack['vulnerability']}")
print(dao_hack['description'])

### What This Cell Does
This cell displays information about the Ronin Bridge Hack (2022), one of the largest crypto exploits ever.

**What to observe:** Description of how North Korean hackers compromised validator keys to steal $625 million.

**Key insight:** This wasn't a code bug — it was a social engineering attack that compromised private keys. Decentralization matters!

In [ ]:
ronin_hack = {
    "date": "March 29, 2022",
    "protocol": "Ronin Bridge (Axie Infinity)",
    "amount_stolen": "$625 million (173,600 ETH + 25.5M USDC)",
    "vulnerability": "Private Key Compromise / Social Engineering",
    "attacker": "Lazarus Group (North Korea)",
    "description": """
Ronin was the Ethereum sidechain for Axie Infinity, a popular blockchain game.

The Attack:
1. Ronin Bridge used 9 validators, requiring 5/9 signatures to approve transactions
2. 4 validators were controlled by Sky Mavis (Axie's developer)
3. A 5th validator key was still authorized from an old arrangement with Axie DAO
4. Attackers (Lazarus Group) compromised all 5 keys through social engineering
5. They approved two fraudulent withdrawal transactions
6. The hack went undetected for 6 DAYS until a user tried to withdraw

Key Failures:
- Excessive centralization (5/9 controlled by related parties)
- Stale permissions not revoked
- No monitoring for unusual transactions
- Social engineering of key holders

Lessons:
- Decentralization matters for security
- Key management is critical
- Real-time monitoring is essential
- Bridges are high-value targets
"""
}

print("\nCASE STUDY: RONIN BRIDGE HACK")
print("=" * 70)
print(f"Date: {ronin_hack['date']}")
print(f"Protocol: {ronin_hack['protocol']}")
print(f"Amount: {ronin_hack['amount_stolen']}")
print(f"Vulnerability: {ronin_hack['vulnerability']}")
print(f"Attacker: {ronin_hack['attacker']}")
print(ronin_hack['description'])

### What This Cell Does
This cell displays information about the Wormhole Bridge Hack (2022).

**What to observe:** Description of how a signature verification bug allowed forging messages to mint tokens without real deposits.

**Key insight:** Cross-chain bridges are complex and vulnerable. A fix had been prepared but not deployed yet!

In [ ]:
wormhole_hack = {
    "date": "February 2, 2022",
    "protocol": "Wormhole Bridge",
    "amount_stolen": "$320 million (120,000 ETH)",
    "vulnerability": "Signature Verification Bug",
    "description": """
Wormhole is a cross-chain bridge connecting Solana to other blockchains.

The Attack:
1. Wormhole used 'guardian' signatures to verify cross-chain messages
2. The Solana program had a bug in signature verification
3. Attacker created a fake 'SignatureSet' account that appeared valid
4. Used this to forge a message claiming 120,000 ETH was deposited
5. Minted 120,000 wETH on Solana without any real deposit
6. Bridged it back to Ethereum and withdrew real ETH

Technical Details:
- The bug was in how 'verify_signatures' validated the signer
- It used a deprecated Solana function that could be spoofed
- A patch had been prepared but not yet deployed!

Resolution:
- Jump Crypto (Wormhole's backer) replaced the stolen funds
- Protocol was paused, audited, and re-secured
- Highlighted the complexity of cross-chain security
"""
}

print("\nCASE STUDY: WORMHOLE BRIDGE HACK")
print("=" * 70)
print(f"Date: {wormhole_hack['date']}")
print(f"Protocol: {wormhole_hack['protocol']}")
print(f"Amount: {wormhole_hack['amount_stolen']}")
print(f"Vulnerability: {wormhole_hack['vulnerability']}")
print(wormhole_hack['description'])

## Section 7: Security Patterns and Best Practices

### Smart Contract Security Patterns

### What This Cell Does
This cell displays a comprehensive guide to smart contract security patterns with code examples.

**What to observe:** Six major security patterns with descriptions and example code showing both vulnerable and secure approaches.

**You don't need to understand the Solidity code — focus on reading the descriptions and understanding the concepts.**

In [ ]:
security_patterns = {
    "Checks-Effects-Interactions": {
        "description": "Always update state before external calls",
        "prevents": "Reentrancy attacks",
        "example": """
// CORRECT:
function withdraw() {
    uint amount = balances[msg.sender];
    require(amount > 0);        // CHECK
    balances[msg.sender] = 0;   // EFFECT (update state first!)
    msg.sender.call{value: amount}("");  // INTERACTION (external call last)
}
"""
    },
    "Reentrancy Guard": {
        "description": "Use a mutex to prevent re-entry",
        "prevents": "Reentrancy attacks",
        "example": """
bool private locked;

modifier nonReentrant() {
    require(!locked, "No reentrancy");
    locked = true;
    _;
    locked = false;
}

function withdraw() nonReentrant {
    // Safe from reentrancy
}
"""
    },
    "Pull Over Push": {
        "description": "Let users withdraw rather than pushing funds to them",
        "prevents": "Denial of service, reentrancy",
        "example": """
// WRONG: Push pattern (risky)
function distribute(address[] recipients) {
    for (uint i = 0; i < recipients.length; i++) {
        recipients[i].call{value: amount}("");  // Can fail or reenter
    }
}

// CORRECT: Pull pattern (safe)
mapping(address => uint) public pendingWithdrawals;

function withdraw() {
    uint amount = pendingWithdrawals[msg.sender];
    pendingWithdrawals[msg.sender] = 0;
    msg.sender.call{value: amount}("");
}
"""
    },
    "Time-Weighted Oracles": {
        "description": "Use TWAP instead of spot prices",
        "prevents": "Flash loan oracle manipulation",
        "example": """
// Use Uniswap V3 TWAP or Chainlink
function getSecurePrice() returns (uint) {
    // Bad: spot price
    // return uniswapPool.slot0.sqrtPriceX96;
    
    // Good: TWAP over 30 minutes
    return uniswapPool.observe(30 minutes);
}
"""
    },
    "Access Control": {
        "description": "Restrict sensitive functions to authorized users",
        "prevents": "Unauthorized access, privilege escalation",
        "example": """
address public owner;
mapping(address => bool) public admins;

modifier onlyOwner() {
    require(msg.sender == owner, "Not owner");
    _;
}

modifier onlyAdmin() {
    require(admins[msg.sender], "Not admin");
    _;
}

function emergencyWithdraw() onlyOwner {
    // Critical function protected
}
"""
    },
    "Timelocks": {
        "description": "Delay sensitive operations to allow intervention",
        "prevents": "Rushed governance attacks, emergency response",
        "example": """
uint public constant TIMELOCK = 2 days;
mapping(bytes32 => uint) public pendingChanges;

function queueChange(bytes32 changeId) onlyAdmin {
    pendingChanges[changeId] = block.timestamp + TIMELOCK;
}

function executeChange(bytes32 changeId) {
    require(pendingChanges[changeId] != 0, "Not queued");
    require(block.timestamp >= pendingChanges[changeId], "Timelock active");
    // Execute change
}
"""
    }
}

print("SMART CONTRACT SECURITY PATTERNS")
print("=" * 70)

for pattern_name, pattern_info in security_patterns.items():
    print(f"\n{pattern_name}")
    print("-" * 40)
    print(f"Description: {pattern_info['description']}")
    print(f"Prevents: {pattern_info['prevents']}")
    print(f"Example: {pattern_info['example']}")

## Section 8: How Security Audits Work

### The Audit Process

### What This Cell Does
This cell displays the complete process that security auditors follow when reviewing smart contracts.

**What to observe:** Six phases from preparation through remediation, with specific tasks in each phase.

**Key insight:** Auditing is systematic and multi-layered, combining automated tools with manual expert review.

In [ ]:
audit_process = {
    "Phase 1: Preparation": [
        "Scope definition - which contracts to audit",
        "Documentation review - specs, architecture docs",
        "Threat modeling - identify potential attack vectors",
        "Test coverage analysis - review existing tests"
    ],
    "Phase 2: Automated Analysis": [
        "Static analysis tools (Slither, Mythril, Securify)",
        "Symbolic execution (Manticore)",
        "Fuzzing (Echidna, Foundry)",
        "Gas optimization analysis"
    ],
    "Phase 3: Manual Review": [
        "Line-by-line code review by security experts",
        "Business logic analysis",
        "Access control verification",
        "Edge case identification",
        "Integration point analysis"
    ],
    "Phase 4: Testing": [
        "Write proof-of-concept exploits for findings",
        "Verify vulnerabilities are exploitable",
        "Test recommended fixes",
        "Regression testing"
    ],
    "Phase 5: Reporting": [
        "Document all findings with severity ratings",
        "Provide remediation recommendations",
        "Executive summary for stakeholders",
        "Technical appendices with details"
    ],
    "Phase 6: Remediation": [
        "Development team fixes issues",
        "Auditors verify fixes",
        "Re-audit if significant changes",
        "Final report publication"
    ]
}

print("SMART CONTRACT AUDIT PROCESS")
print("=" * 70)

for phase, steps in audit_process.items():
    print(f"\n{phase}")
    print("-" * 40)
    for step in steps:
        print(f"  - {step}")

### What This Cell Does
This cell displays how audit findings are classified by severity level (Critical, High, Medium, Low, Informational).

**What to observe:** Each severity level with description, examples, and recommended actions.

**Key insight:** Not all security issues are equally urgent. Critical issues must be fixed before deployment.**

In [ ]:
# Common audit findings and severity levels
severity_levels = {
    "Critical": {
        "description": "Direct loss of funds or complete contract compromise",
        "examples": [
            "Reentrancy allowing fund drainage",
            "Missing access control on withdrawal",
            "Integer overflow allowing infinite minting"
        ],
        "action": "Must fix before deployment"
    },
    "High": {
        "description": "Significant risk to funds or functionality",
        "examples": [
            "Oracle manipulation vulnerability",
            "Front-running opportunities",
            "Privilege escalation"
        ],
        "action": "Should fix before deployment"
    },
    "Medium": {
        "description": "Moderate risk or significant edge cases",
        "examples": [
            "Missing input validation",
            "Centralization risks",
            "Incomplete event logging"
        ],
        "action": "Fix recommended"
    },
    "Low": {
        "description": "Minor issues or best practice violations",
        "examples": [
            "Gas optimizations",
            "Code style issues",
            "Missing comments"
        ],
        "action": "Consider fixing"
    },
    "Informational": {
        "description": "Observations and suggestions",
        "examples": [
            "Architecture recommendations",
            "Testing suggestions",
            "Documentation improvements"
        ],
        "action": "For consideration"
    }
}

print("\nAUDIT FINDING SEVERITY LEVELS")
print("=" * 70)

for level, info in severity_levels.items():
    print(f"\n{level}")
    print("-" * 40)
    print(f"Description: {info['description']}")
    print(f"Action: {info['action']}")
    print(f"Examples:")
    for ex in info['examples']:
        print(f"  - {ex}")

### What This Cell Does
This cell displays a reference guide to audit firms, automated security tools, bug bounty platforms, and resources.

**What to observe:** Lists of top audit firms, security tools, bug bounty platforms, and learning resources.

**You might want to:** Bookmark some of these resources for later exploration (like Rekt.news for exploit analysis).

In [ ]:
# Top audit firms and tools
audit_ecosystem = {
    "Top Audit Firms": [
        "Trail of Bits - Known for Slither, Echidna, Manticore",
        "OpenZeppelin - Standard library, audit services",
        "Consensys Diligence - Mythril, MythX",
        "Sigma Prime - Ethereum 2.0, Lighthouse",
        "Halborn - Multi-chain expertise",
        "Quantstamp - Automated + manual audits",
        "Certik - Formal verification focus",
        "Spearbit - Elite security researchers"
    ],
    "Automated Tools": [
        "Slither - Static analysis for Solidity",
        "Mythril - Symbolic execution for EVM",
        "Echidna - Property-based fuzzer",
        "Foundry - Testing framework with fuzzing",
        "Manticore - Symbolic execution engine",
        "Securify - Security scanner",
        "Oyente - Symbolic execution (older)"
    ],
    "Bug Bounty Platforms": [
        "Immunefi - Largest crypto bug bounty platform",
        "HackerOne - General security, includes Web3",
        "Code4rena - Competitive audits",
        "Sherlock - Audit marketplace"
    ],
    "Security Resources": [
        "SWC Registry - Smart Contract Weakness Classification",
        "Rekt.news - DeFi exploit analysis",
        "DeFi Safety - Protocol safety scores",
        "OpenZeppelin Contracts - Audited standard library"
    ]
}

print("\nDEFI SECURITY ECOSYSTEM")
print("=" * 70)

for category, items in audit_ecosystem.items():
    print(f"\n{category}")
    print("-" * 40)
    for item in items:
        print(f"  - {item}")

## Challenge 1: Identify Vulnerabilities (Conceptual)

Below are descriptions of four code patterns. Based on what you've learned, identify which vulnerability each one has.

**Pattern A: Token Vault**
- A contract that holds user funds
- Has a `withdraw()` function that: (1) checks the user's balance, (2) sends ETH to the user, (3) updates the balance to zero

**Pattern B: Price Oracle**
- A lending protocol that checks collateral value
- Gets the current price by dividing two numbers from a DEX: `price = reserve1 / reserve0`
- Uses this price to decide if a user can borrow more money

**Pattern C: Admin Functions**
- A treasury contract with an owner address
- Has a `setOwner()` function that anyone can call to change the owner
- Has a `withdrawAll()` function that only the owner can call

**Pattern D: Token Transfer**
- A token contract that tracks balances
- Has a `transfer()` function that: (1) subtracts tokens from sender, (2) adds tokens to recipient
- Written in old Solidity (version 0.7) without overflow protection

---

### Questions (Answer these based on what you've learned):

**Q1:** Which pattern has a reentrancy vulnerability? What's the problem?

**Q2:** Which pattern has an oracle manipulation vulnerability? Why is it vulnerable?

**Q3:** Which pattern has a missing access control vulnerability? What should be added?

**Q4:** Which pattern has an integer underflow vulnerability? What happens if someone tries to transfer more tokens than they have?

---

### Reflection Questions:

**Q5:** Based on the exploit data you saw earlier, which of these four vulnerabilities has caused the most financial losses in real DeFi protocols?

**Q6:** If you were a security auditor reviewing these patterns, which ONE would you flag as "Critical" severity and require fixing before deployment? Why?

**Q7:** Looking at the reentrancy simulation earlier, what TWO defensive techniques did the secure bank use to prevent the attack?

## Challenge 2: Design Security for a Lending Protocol (Conceptual)

Imagine you're advising a new DeFi lending protocol. The protocol will allow users to:
1. Deposit collateral (ETH or tokens)
2. Borrow stablecoins against collateral
3. Liquidate undercollateralized positions

Based on what you've learned in this notebook, answer these questions:

---

### Price Oracle Questions:

**Q1:** Should the protocol use spot prices or TWAP (Time-Weighted Average Price)? Why?

**Q2:** What could go wrong if the protocol gets prices from only ONE source (like a single DEX)?

**Q3:** If you notice the price suddenly jumps 50% in a single block, what should the protocol do?

---

### Borrowing & Withdrawal Questions:

**Q4:** Should users be able to deposit collateral AND borrow in the SAME transaction (same block)? Why or why not?

**Q5:** When a user withdraws their collateral, should the protocol send the money BEFORE or AFTER updating the user's balance? Why?

**Q6:** What security pattern should EVERY withdrawal function use? (Hint: Remember the reentrancy section!)

---

### Access Control Questions:

**Q7:** The protocol has an emergency pause function. Who should be able to call it?
   - (A) Only one person
   - (B) A multisig wallet (requires 3 out of 5 signatures)
   - (C) Anyone

   Choose one and explain your reasoning.

**Q8:** If the protocol needs to upgrade its smart contracts, should the upgrade happen:
   - (A) Immediately when requested
   - (B) After a 2-day timelock delay
   - (C) Never (immutable contracts)

   What are the tradeoffs?

---

### Risk Assessment Questions:

**Q9:** Based on the exploit data you saw, what is the HIGHEST risk category for this protocol?
   - Smart contract bugs
   - Oracle manipulation
   - Private key compromise
   - Frontend attacks

**Q10:** If you had budget for only TWO security measures, which would you prioritize?
   - (A) Smart contract audit by a top firm
   - (B) Bug bounty program
   - (C) TWAP price oracle
   - (D) Insurance fund
   - (E) 24/7 monitoring system

   Choose TWO and explain why.

## Challenge 3: Analyze Historical Exploit Patterns

Look back at the exploit data from earlier in the notebook and answer these analysis questions:

---

### Trend Analysis:

**Q1:** Looking at the "Losses by Year" table, describe the trend from 2016 to 2023. Are exploits getting more or less severe over time?

**Q2:** The year 2022 was particularly bad for exploits. Based on the data, what TYPE of attack caused the most losses in 2022? (Hint: Look at the exploit list)

**Q3:** Bridge attacks (like Ronin, Wormhole, Nomad, BNB Bridge) account for what percentage of total losses? Why do you think bridges are such attractive targets?

---

### Attack Pattern Recognition:

**Q4:** Compare these three attacks:
   - The DAO (2016): Reentrancy - $60M
   - Curve Finance (2023): Reentrancy (Vyper bug) - $70M

   Reentrancy was a well-known vulnerability in 2016. Why do you think it still caused a major exploit 7 years later?

**Q5:** Flash Loan attacks appear many times in the list (bZx, Yearn, DODO, Spartan, PancakeBunny, Cream, Euler). What do these attacks have in common? Why can't they be executed without flash loans?

**Q6:** Looking at the attacker field in the Ronin case study (North Korea's Lazarus Group), what does this tell you about the changing nature of crypto exploits?

---

### Defense Prioritization:

**Q7:** Based on the data, rank these three defense strategies from MOST to LEAST important:
   - (A) Preventing reentrancy attacks
   - (B) Securing cross-chain bridges
   - (C) Using manipulation-resistant oracles

   Explain your ranking using evidence from the exploit data.

**Q8:** If you were building a new DeFi protocol in 2024, which historical exploit would you study MOST carefully to avoid repeating mistakes? Why?

**Q9:** Many of the exploited protocols (like Cream Finance, Beanstalk, Euler) had been audited before being hacked. What does this tell you about the limitations of audits?

**Q10:** Timeline question: How many days passed between the Ronin Bridge hack occurring (March 29, 2022) and it being DETECTED? (Hint: The case study mentions this). What does this reveal about the importance of monitoring?

## Summary

In this notebook, you learned about DeFi security from a **defensive perspective**:

### Key Concepts:

1. **DeFi Security Landscape**
   - Over $3 billion lost to exploits since 2016
   - Major attack categories: smart contract bugs, economic exploits, infrastructure attacks
   - Bridges and flash loans are particularly risky attack surfaces

2. **Reentrancy Attacks**
   - Occur when external calls happen before state updates
   - Prevention: Checks-Effects-Interactions pattern + reentrancy guards
   - The DAO hack (2016) was the first major example

3. **Flash Loan Mechanics**
   - Uncollateralized loans that must be repaid in same transaction
   - Enable both legitimate arbitrage and malicious attacks
   - Often combined with oracle manipulation

4. **Oracle Manipulation**
   - Spot prices are easily manipulable
   - TWAP and multiple sources provide resistance
   - Critical for any protocol using external price data

5. **Security Best Practices**
   - Checks-Effects-Interactions pattern
   - Reentrancy guards
   - Access control
   - Timelocks
   - Multiple audits
   - Bug bounties

6. **Audit Process**
   - Automated tools + manual review
   - Severity classification
   - Continuous security (not one-time)

### Key Takeaways:

- **Security is not optional** in DeFi - bugs mean lost funds
- **Multiple layers of defense** are necessary
- **Audits are necessary but not sufficient** - many hacked protocols were audited
- **Economic security** is as important as code security
- **Monitoring and incident response** are critical

### Further Reading:

- [SWC Registry](https://swcregistry.io/) - Smart Contract Weakness Classification
- [Rekt.news](https://rekt.news/) - DeFi exploit analysis
- [Immunefi](https://immunefi.com/) - Bug bounty platform
- [Trail of Bits Blog](https://blog.trailofbits.com/) - Security research
- [OpenZeppelin Contracts](https://docs.openzeppelin.com/contracts/) - Audited standard library
- [Damn Vulnerable DeFi](https://www.damnvulnerabledefi.xyz/) - Security challenges